In [2]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import findspark
findspark.init()
import pyspark as spark
from pyspark.sql.functions import rand,udf,col
from pyspark.sql.types import StringType
sc = spark.SparkContext()
sql=spark.SQLContext(sc)

In [8]:
df = pd.read_csv("/home/fvalle/GoogleDrive/tesi_magistrale/tesi/gtex/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_reads.gct", sep='\t')
df.index = [x[:15] for x in df['Name']]

In [4]:
df_file=sql.read.option('delimiter',',').option("header",'true').csv("/home/fvalle/GoogleDrive/tesi_magistrale/tesi/gtex/files.dat.ok")
df_file.registerTempTable("files")
available_tissues=df_file.groupby('primary_site').count().sort("count", ascending=False).toPandas()

In [5]:
directory=r"/home/fvalle/GoogleDrive/tesi_magistrale/tesi/gtex/123experiments/"
os.chdir(directory)

In [7]:
model = PCA()
intr_dim={
    1e10:[],
    2e10:[],
    2.1e10:[],
    2.3e10:[],
    2.5e10:[]
}
x=np.arange(2,15,step=1, dtype=int)
xt=[]
for i in x:
    print(i)
    selectedTissues = available_tissues['primary_site'][:].values[:i]
    needed_columns = []
    for tissue in selectedTissues:
        needed_columns = np.concatenate((needed_columns,
                                     list(set(sql.sql("SELECT * FROM files WHERE primary_site='%s'"%(tissue))
                                    .orderBy(rand())
                                    .select("_c0")
                                    .toPandas().values.T[0])-{'Name'}-{'Description'})))
    needed_columns=needed_columns[np.isin(needed_columns,df.columns)]
    needed_columns = np.concatenate([['Name'],needed_columns])
    df_nt=df[df.columns.isin(needed_columns)]
    try:
        #df = pd.read_csv("experiment%d/mainTable.csv"%i, index_col=0)
        df_nt.values.T.shape
        model.fit(df_nt.values.T[:1500])
        var = model.explained_variance_
        for k in intr_dim.keys():
            intrsize=len(var[var>k])
            intr_dim[k].append(intrsize)
        xt.append(i)
    except:
        print(sys.exc_info()[1])

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/spark-2.4.4-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


2


Py4JError: An error occurred while calling o21.sql

In [ ]:
fig=plt.figure()
for k in intr_dim.keys():
    plt.plot(xt,intr_dim[k], label=k,  ls='-', marker='o', ms=5)
plt.plot(xt,xt,c='blue', ls='--')
plt.legend()
plt.show()

In [ ]:
sc.stop()